In [1]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
path = '/projectnb2/cs505/yizh'

In [2]:
def getFrame(sec,count,filename):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    stroing_path = '/projectnb2/cs505/yizh/pj_data' #place to store png
    hasFrames,image = vidcap.read()
    if hasFrames:
#         cropped = image[8:136, 26:154]
#         dim = (256, 256)

#         resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(stroing_path,'image'+str(0)+".png"), image)     # save frame as JPG file
    return hasFrames

In [15]:
# filename = 'background.avi'
# vidcap = cv2.VideoCapture(path+'/'+filename)
# success = getFrame(0,0)

In [6]:
import cv2
count = 0
for filename in os.listdir(path+'/bg'):

    img_count = 0
    #if filename.startswith('.'):
    #    os.remove(path+'/'+filename)
    vidcap = cv2.VideoCapture(path+'/bg/'+filename)
    sec = 0
    frameRate = 0.2 #//it will capture image in each 0.5 second
    success = getFrame(sec,count,filename)
    count += 1
    img_count += 1
    while success and img_count < 1:
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec,count,filename)
        count += 1
        img_count += 1


In [8]:
img_path = '/projectnb2/cs505/yizh/pj_data'


files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]
files.sort()
img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))



In [10]:
def find_center(c):

    M = cv2.moments(c)
    if M['m00'] == 0:
        cX = int(c[0][0][0])
        cY = int(c[0][0][1])
    else:
        cX = int(M['m10']/M['m00'])
        cY = int(M['m01']/M['m00'])

    return cX,cY

def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    '''
    dst = cv2.absdiff(prev, curr)
    gs = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    dst = (gs > 20).astype(np.uint8) * 255
    return dst

In [16]:
stroing_path = '/projectnb2/cs505/yizh/pj_data'
bg = cv2.imread('/projectnb2/cs505/yizh/pj_data/image0.png')

w,h=bg.shape[1],bg.shape[0]
dim = (w*4,h*4)
bg = cv2.resize(bg, dim, interpolation = cv2.INTER_AREA)

img_indx = 0
count = 1
while img_indx < len(files):
# while img_indx < 50:
    cur = img[img_indx]
    cur = cv2.resize(cur, dim, interpolation = cv2.INTER_AREA)
    
    #print(img_indx)
    diff = my_frame_differencing(bg, cur)

    # gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    thres_output = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,207,-5)
    contours, hierarchy = cv2.findContours(thres_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    if (len(contours) > 0):
        # Find largest contour
        max_id = max(enumerate(contours), key=lambda x : cv2.contourArea(x[1]))[0]
        max_size = cv2.contourArea(contours[max_id])
        boundrec = cv2.boundingRect(contours[max_id])
        cv2.drawContours(cur, contours, max_id, (255, 255, 255), 2, 8)
        cv2.rectangle(cur, boundrec, (0, 255, 0), 1, 8, 0)
        X,Y = find_center(contours[max_id])
        #print(X,Y)


    #cropped = cur[Y-200:Y+200,X-200:X+200]
    #cropped = cur = cv2.resize(cropped, (256,256), interpolation = cv2.INTER_AREA)
    



    cv2.imwrite(os.path.join(stroing_path,'image'+str(count)+".png"), cur)     # save frame as JPG file
    
    count += 1
    img_indx += 1
    
    
    



In [84]:
cropped.shape

(384, 384, 3)